In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("datasets/perovai_devices_day4_rawLIV.csv")
df.columns = df.columns.str.strip()
df.head()

,Voc_V,Jsc_mAcm2,FF,PCE_percent,Rs_ohm_cm2,Rsh_ohm_cm2,Device,Scan,File
0,1.113055,20.178596,0.640161,14.377952,8.304344,4474.454512,device001,Reverse,device001_LIV1.csv
1,1.104883,20.177047,0.650051,14.491762,8.245944,5106.928620,device001,Forward,device001_LIV2.csv
2,1.162480,20.154208,0.758766,17.777031,7.594258,7605.396653,device002,Reverse,device002_LIV1.csv
3,1.144383,20.051937,0.746704,17.134683,6.118376,3341.636255,device002,Forward,device002_LIV2.csv
4,1.152423,20.797109,0.767192,18.387340,5.584745,8414.256139,device003,Reverse,device003_LIV1.csv


In [3]:
print("Total rows:", len(df))
print(df["Scan"].value_counts())
print("Unique devices:", df["Device"].nunique())

Total rows: 212
Scan
Reverse    106
Forward    106
Name: count, dtype: int64
Unique devices: 106


In [4]:
pivot = df.pivot_table(
    index="Device",
    columns="Scan",
    values=[
        "Voc_V",
        "Jsc_mAcm2",
        "FF",
        "PCE_percent",
        "Rs_ohm_cm2",
        "Rsh_ohm_cm2"
    ],
    aggfunc="mean"
)

pivot

FF            Jsc_mAcm2            PCE_percent             \
Scan        Forward   Reverse    Forward    Reverse     Forward    Reverse   
Device                                                                       
device001  0.650051  0.640161  20.177047  20.178596   14.491762  14.377952   
device002  0.746704  0.758766  20.051937  20.154208   17.134683  17.777031   
device003  0.771115  0.767192  20.756446  20.797109   18.363962  18.387340   
device004  0.748184  0.759476  20.071712  20.171704   17.191299  17.810883   
device005  0.748388  0.755708  20.152430  20.259536   17.333143  17.911775   
...             ...       ...        ...        ...         ...        ...   
device102  0.455451  0.461271  18.178478  18.351961    6.791652   7.139479   
device103  0.688716  0.660341  19.130148  19.191203   14.709928  14.272135   
device104  0.680467  0.665765  18.360162  18.427811   13.924797  13.736898   
device105  0.243488  0.243987   9.733354   9.790709    0.034593   0.035090   
device106  0.567340  0.556037  18.192962  18.254312   10.882210  10.830864   

          Rs_ohm_cm2             Rsh_ohm_cm2                  Voc_V            
Scan         Forward    Reverse      Forward      Reverse   Forward   Reverse  
Device                                                                         
device001   8.245944   8.304344  5106.928620  4474.454512  1.104883  1.113055  
device002   6.118376   7.594258  3341.636255  7605.396653  1.144383  1.162480  
device003   4.786209   5.584745  4110.186343  8414.256139  1.147345  1.152423  
device004   6.110401   7.581881  3240.772843  7279.252937  1.144763  1.162596  
device005   6.452673   8.205573  3485.020804  8596.770368  1.149273  1.169918  
...              ...        ...          ...          ...       ...       ...  
device102  13.594969  13.283675   135.768773   157.085418  0.820306  0.843388  
device103   7.593493   8.752211  2992.938351  2660.734683  1.116482  1.126207  
device104   7.773186   8.590910  2595.819207  4018.012413  1.114565  1.119679  
device105   1.499723   1.500849     1.499061     1.500024  0.014597  0.014689  
device106  12.152187  12.000839   316.643628   365.375451  1.054315  1.067072  

[106 rows x 12 columns]

In [5]:
pivot.columns = [f"{p}_{s}" for p, s in pivot.columns]
pivot = pivot.reset_index()
pivot.head()

,Device,FF_Forward,FF_Reverse,Jsc_mAcm2_Forward,Jsc_mAcm2_Reverse,PCE_percent_Forward,PCE_percent_Reverse,Rs_ohm_cm2_Forward,Rs_ohm_cm2_Reverse,Rsh_ohm_cm2_Forward,Rsh_ohm_cm2_Reverse,Voc_V_Forward,Voc_V_Reverse
0,device001,0.650051,0.640161,20.177047,20.178596,14.491762,14.377952,8.245944,8.304344,5106.928620,4474.454512,1.104883,1.113055
1,device002,0.746704,0.758766,20.051937,20.154208,17.134683,17.777031,6.118376,7.594258,3341.636255,7605.396653,1.144383,1.162480
2,device003,0.771115,0.767192,20.756446,20.797109,18.363962,18.387340,4.786209,5.584745,4110.186343,8414.256139,1.147345,1.152423
3,device004,0.748184,0.759476,20.071712,20.171704,17.191299,17.810883,6.110401,7.581881,3240.772843,7279.252937,1.144763,1.162596
4,device005,0.748388,0.755708,20.152430,20.259536,17.333143,17.911775,6.452673,8.205573,3485.020804,8596.770368,1.149273,1.169918


In [6]:
pivot["Delta_PCE"] = pivot["PCE_percent_Reverse"] - pivot["PCE_percent_Forward"]
pivot["Delta_Voc"] = pivot["Voc_V_Reverse"] - pivot["Voc_V_Forward"]
pivot["Delta_Jsc"] = pivot["Jsc_mAcm2_Reverse"] - pivot["Jsc_mAcm2_Forward"]
pivot["Delta_FF"]  = pivot["FF_Reverse"] - pivot["FF_Forward"]

# avoid divide-by-zero
pivot["Hysteresis_Index"] = pivot["Delta_PCE"] / pivot["PCE_percent_Reverse"].replace(0, np.nan)
pivot.head()

,Device,FF_Forward,FF_Reverse,Jsc_mAcm2_Forward,Jsc_mAcm2_Reverse,PCE_percent_Forward,PCE_percent_Reverse,Rs_ohm_cm2_Forward,Rs_ohm_cm2_Reverse,Rsh_ohm_cm2_Forward,Rsh_ohm_cm2_Reverse,Voc_V_Forward,Voc_V_Reverse,Delta_PCE,Delta_Voc,Delta_Jsc,Delta_FF,Hysteresis_Index
0,device001,0.650051,0.640161,20.177047,20.178596,14.491762,14.377952,8.245944,8.304344,5106.928620,4474.454512,1.104883,1.113055,-0.113810,0.008172,0.001549,-0.009890,-0.007916
1,device002,0.746704,0.758766,20.051937,20.154208,17.134683,17.777031,6.118376,7.594258,3341.636255,7605.396653,1.144383,1.162480,0.642347,0.018098,0.102270,0.012062,0.036134
2,device003,0.771115,0.767192,20.756446,20.797109,18.363962,18.387340,4.786209,5.584745,4110.186343,8414.256139,1.147345,1.152423,0.023378,0.005078,0.040663,-0.003923,0.001271
3,device004,0.748184,0.759476,20.071712,20.171704,17.191299,17.810883,6.110401,7.581881,3240.772843,7279.252937,1.144763,1.162596,0.619584,0.017833,0.099991,0.011292,0.034787
4,device005,0.748388,0.755708,20.152430,20.259536,17.333143,17.911775,6.452673,8.205573,3485.020804,8596.770368,1.149273,1.169918,0.578632,0.020645,0.107106,0.007320,0.032305


In [7]:
pivot_sorted = pivot.sort_values("Hysteresis_Index", key=np.abs, ascending=False)

pivot_sorted[[
    "Device",
    "PCE_percent_Reverse",
    "PCE_percent_Forward",
    "Delta_PCE",
    "Hysteresis_Index"
]].head(25)

,Device,PCE_percent_Reverse,PCE_percent_Forward,Delta_PCE,Hysteresis_Index
86,device087,4.727265e-06,-8.551959e-03,8.556686e-03,1810.071312
74,device075,-5.232201e-02,2.860636e-07,-5.232229e-02,1.000005
78,device079,7.174571e-06,2.287931e-07,6.945778e-06,0.968111
94,device095,5.021629e-06,3.581008e-07,4.663528e-06,0.928688
98,device099,6.017014e-06,4.506896e-07,5.566325e-06,0.925097
47,device048,5.871202e-06,5.456964e-07,5.325505e-06,0.907055
99,device100,4.750336e-06,4.803376e-07,4.269998e-06,0.898883
96,device097,4.585688e-06,4.860912e-07,4.099596e-06,0.893998
90,device091,3.385512e-06,3.726139e-07,3.012898e-06,0.889939
92,device093,4.775795e-06,5.517884e-07,4.224006e-06,0.884461


In [8]:
pivot["High_Hysteresis"] = (pivot["Hysteresis_Index"].abs() > 0.05).astype(int)

# median-based flag for quick stability proxy later
pivot["Low_Rs"] = (pivot["Rs_ohm_cm2_Reverse"] < pivot["Rs_ohm_cm2_Reverse"].median()).astype(int)

pivot[["Device","High_Hysteresis","Low_Rs"]].head()

,Device,High_Hysteresis,Low_Rs
0,device001,0,1
1,device002,0,1
2,device003,0,1
3,device004,0,1
4,device005,0,1


In [9]:
pivot.to_csv("datasets/perovai_devices_day5_hysteresis.csv", index=False)
print("Saved: datasets/perovai_devices_day5_hysteresis.csv")
print("Rows (devices):", len(pivot))

Saved: datasets/perovai_devices_day5_hysteresis.csv
Rows (devices): 106


In [10]:
missing_rev = pivot["PCE_percent_Reverse"].isna().sum()
missing_fwd = pivot["PCE_percent_Forward"].isna().sum()
print("Missing Reverse PCE:", missing_rev)
print("Missing Forward PCE:", missing_fwd)

Missing Reverse PCE: 0
Missing Forward PCE: 0
